# This notebook creates clips which contain gunshots for the testing data, obtained from long 24 hours audios. 

In [1]:
import wave

import pandas as pd
from pydub import AudioSegment

/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python

Read in data frames

In [2]:
korup = pd.read_csv("/Users/rosameliacarioni/University/Thesis/code/data_frames/test/korup.csv")
pnnn_1 = pd.read_csv("/Users/rosameliacarioni/University/Thesis/code/data_frames/test/pnnn_1.csv")
pnnn_2 = pd.read_csv("/Users/rosameliacarioni/University/Thesis/code/data_frames/test/pnnn_2.csv")
pnnn_3 = pd.read_csv("/Users/rosameliacarioni/University/Thesis/code/data_frames/test/pnnn_3.csv")
pnnn_4 = pd.read_csv("/Users/rosameliacarioni/University/Thesis/code/data_frames/test/pnnn_4.csv")

In [3]:
korup

,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Begin File,File Offset (s),delta time (s)
0,5.573217e+03,5.575591e+03,0.000,956.000,kp01_20130621_000000.wav,5573.2167,2.374362
1,1.053806e+04,1.054070e+04,0.000,986.000,kp01_20130621_000000.wav,10538.0623,2.638180
2,2.377667e+04,2.377999e+04,0.000,988.000,kp01_20130621_000000.wav,23776.6702,3.316570
3,2.386634e+04,2.386924e+04,38.000,930.000,kp01_20130621_000000.wav,23866.3400,2.902000
4,2.388696e+04,2.388907e+04,28.000,832.000,kp01_20130621_000000.wav,23886.9568,2.115210
...,...,...,...,...,...,...,...
704,8.226388e+06,8.226389e+06,88.372,732.558,kp15_20150422_000000.wav,46884.8070,0.992000
705,8.226502e+06,8.226503e+06,51.163,725.581,kp15_20150422_000000.wav,46998.5612,1.412000
706,8.226510e+06,8.226511e+06,69.767,725.581,kp15_20150422_000000.wav,47006.5687,0.918000
707,8.226512e+06,8.226513e+06,79.070,720.930,kp15_20150422_000000.wav,47008.5925,0.992000


# Transform audios to all have a sample rate of 8.000 

In [ ]:
# Time is passed in seconds


def change_one_file_rate(file):
    file_location = "/Volumes/Meli_disk/thesis_data/Sounds_ELP/" + file
    new_sample_rate = 8000
    try:
        audio = AudioSegment.from_wav(file_location)
        audio = audio.set_frame_rate(new_sample_rate)
        audio.export(file_location, format="wav")
    except FileNotFoundError:
        print("ERROR ", file_location)


def change_all_files(files):
    for file in files:
        change_one_file_rate(file)

Resample all files needed/used by the different data frames

In [ ]:
files_korup = korup["Begin File"].unique().tolist()
change_all_files(files_korup)

In [ ]:
files_pnnn_1 = pnnn_1["Begin File"].unique().tolist()
change_all_files(files_pnnn_1)

In [ ]:
files_pnnn_2 = pnnn_2["Begin File"].unique().tolist()
change_all_files(files_pnnn_2)

In [ ]:
files_pnnn_3 = pnnn_3["Begin File"].unique().tolist()
change_all_files(files_pnnn_3)

# Clip sounds containing gunshots

In [3]:
# Time is passed in seconds


def clip_audio_and_save(instance_of_df, extra_name):
    time_start = instance_of_df["File Offset (s)"]
    time_duration = instance_of_df["delta time (s)"]
    file_location = "/Volumes/Meli_disk/thesis_data/Sounds_ELP/" + instance_of_df["Begin File"]
    file_name = "/Volumes/Meli_disk/thesis_data/test/Clips_ELP/gunshot/" + extra_name + str(instance_of_df.name) + ".wav"

    second_to_milliseconds = 1000

    try:
        audio = AudioSegment.from_wav(file_location)
    except FileNotFoundError:
        return None

    clip = audio[
        time_start * second_to_milliseconds : time_start * second_to_milliseconds + time_duration * second_to_milliseconds
    ]
    clip.export(file_name, format="wav")
    return file_name


def clip_all_df(df, extra_name):
    files_not_found = set()
    df["Clip Location"] = df.apply(clip_audio_and_save, axis=1, extra_name=extra_name)

    for index, row in df.iterrows():
        if row["Clip Location"] is None:
            files_not_found.add(row["Begin File"])
    return files_not_found

Call method to generate the clips from the 5 data frames 

In [4]:
korup_not_found = clip_all_df(korup, "korup_")
print(korup_not_found)

set()


In [5]:
one_not_found = clip_all_df(pnnn_1, "pnnn_1_")
print(one_not_found)

set()


In [6]:
two_not_found = clip_all_df(pnnn_2, "pnnn_2_")
print(two_not_found)

set()


In [7]:
three_not_found = clip_all_df(pnnn_3, "pnnn_3_")
print(three_not_found)

set()


In [8]:
four_not_found = clip_all_df(pnnn_4, "pnnn_4_")
print(four_not_found)

set()
